 DAEDALUS – Distributed and Automated Evolutionary Deep Architecture Learning with Unprecedented Scalability

This research code was developed as part of the research programme Open Technology Programme with project number 18373, which was financed by the Dutch Research Council (NWO), Elekta, and Ortec Logiqcare.

Project leaders: Peter A.N. Bosman, Tanja Alderliesten
Researchers: Alex Chebykin, Arthur Guijt, Vangelis Kostoulas
Main code developer: Arthur Guijt

In [1]:
import pandas as pd
import numpy as np

from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

## Load event data

In [26]:
# folder = "../build/results/test-sim-gomea"
# folder = "../build/results/test-sim-synchronous-gomea"
# folder = "../build/results/ecga-sync"
# folder = "../build/results/ecga-async"
# folder = "../build/results/ecga-sync-cpulim"
# folder = "../build/results/ecga-async-cpulim"
# folder = "../build/results/gomea-sync-cpulim"
# folder = "../build/results/gomea-async-cpulim"
# folder = "../build/results/sga-sync"
folder = "../build/results/sga-async"

In [27]:
events = pd.read_json(f"{folder}/events.jsonl", lines=True)
events

,kind,ord,t,desc,ord_before,ord_after
0,new,0,0.0000,Evaluate initial solution 0,NaN,NaN
1,new,1,0.0000,Evaluate initial solution 1,NaN,NaN
2,new,2,0.0000,Evaluate initial solution 2,NaN,NaN
3,new,3,0.0000,Evaluate initial solution 3,NaN,NaN
4,new,4,0.0000,Evaluate initial solution 4,NaN,NaN
...,...,...,...,...,...,...
6331,performed,3196,27.5446,NaN,3197.0,3198.0
6332,performed,3101,27.5485,NaN,3198.0,3198.0
6333,new,3198,27.5485,New solution on 14,NaN,NaN
6334,new,3199,27.5485,Evaluate solution on 14,NaN,NaN


In [28]:
merged = pd.merge(events[events["kind"] == "new"], events[events["kind"] == "performed"], on="ord")
merged["sol"] = merged["desc_x"].str.extract("on ([0-9]+)").astype(int)

merged["c"] = merged["desc_x"].str.extract("element ([0-9]+)").astype(float)
merged["t"] = merged["desc_x"].str.extract("(FI|GOM)").fillna("Init").astype(str)
merged

,kind_x,ord,t_x,desc_x,ord_before_x,ord_after_x,kind_y,t_y,desc_y,ord_before_y,ord_after_y,sol,c,t
0,new,0,0.0000,Evaluate initial solution 0,NaN,NaN,performed,1.277930,NaN,166.0,166.0,0,NaN,Init
1,new,1,0.0000,Evaluate initial solution 1,NaN,NaN,performed,0.910061,NaN,90.0,90.0,1,NaN,Init
2,new,2,0.0000,Evaluate initial solution 2,NaN,NaN,performed,1.267810,NaN,164.0,164.0,2,NaN,Init
3,new,3,0.0000,Evaluate initial solution 3,NaN,NaN,performed,1.158110,NaN,140.0,140.0,3,NaN,Init
4,new,4,0.0000,Evaluate initial solution 4,NaN,NaN,performed,1.067500,NaN,116.0,116.0,4,NaN,Init
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,new,3190,27.4938,New solution on 46,NaN,NaN,performed,27.493800,NaN,3191.0,3192.0,46,NaN,Init
3132,new,3192,27.5133,New solution on 19,NaN,NaN,performed,27.513300,NaN,3193.0,3194.0,19,NaN,Init
3133,new,3194,27.5306,New solution on 17,NaN,NaN,performed,27.530600,NaN,3195.0,3196.0,17,NaN,Init
3134,new,3196,27.5446,New solution on 7,NaN,NaN,performed,27.544600,NaN,3197.0,3198.0,7,NaN,Init


## Load Archive

In [29]:
archive = pd.read_csv(f"{folder}/archive.csv", dtype={"archive ordinals removed": str})
per_objective = archive["objectives"].astype(str).str.split(expand=True)

archive[[f"o{i}" for i in range(per_objective.shape[1])]] = archive["objectives"].astype(str).str.split(expand=True).astype(float)
archive

,#evaluations,simulation time (s),objectives,archive ordinal,archive ordinals removed,genotype (categorical),o0
0,64,0.800467,-11,1,NaN,1 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 ...,-11.0
1,65,0.804240,-12,2,1,0 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 ...,-12.0
2,66,0.807959,-15,3,2,1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 1 0 ...,-15.0
3,71,0.833847,-17,4,3,0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 ...,-17.0
4,97,1.123815,-23,5,4,0 1 1 0 1 1 0 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 0 ...,-23.0
5,384,5.991524,-24,6,5,1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 ...,-24.0
6,1036,17.312336,-25,7,6,0 1 1 1 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 0 0 1 1 ...,-25.0
7,1382,23.229398,-26,8,7,1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 ...,-26.0


In [30]:
# Compute & append front removal time.
archivec = archive.copy()
archivec["removes"] = archivec["archive ordinals removed"].fillna("").str.split()
removals = archivec.explode("removes")[["simulation time (s)", "removes"]]
removals = removals[~removals["removes"].isna()]
removals["removes"] = removals["removes"].astype(int)
archive_wrt = pd.merge(archive, removals, left_on="archive ordinal", right_on="removes", how="left").drop(columns=["removes"])
archive_wrt["start simulation time (s)"] = archive_wrt["simulation time (s)_x"]
archive_wrt["end simulation time (s)"] = archive_wrt["simulation time (s)_y"]
archive_wrt

,#evaluations,simulation time (s)_x,objectives,archive ordinal,archive ordinals removed,genotype (categorical),o0,simulation time (s)_y,start simulation time (s),end simulation time (s)
0,64,0.800467,-11,1,NaN,1 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 ...,-11.0,0.804240,0.800467,0.804240
1,65,0.804240,-12,2,1,0 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 ...,-12.0,0.807959,0.804240,0.807959
2,66,0.807959,-15,3,2,1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 1 0 ...,-15.0,0.833847,0.807959,0.833847
3,71,0.833847,-17,4,3,0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 ...,-17.0,1.123815,0.833847,1.123815
4,97,1.123815,-23,5,4,0 1 1 0 1 1 0 1 1 1 0 1 0 1 0 1 0 0 1 1 1 1 0 ...,-23.0,5.991524,1.123815,5.991524
5,384,5.991524,-24,6,5,1 1 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 ...,-24.0,17.312336,5.991524,17.312336
6,1036,17.312336,-25,7,6,0 1 1 1 0 0 0 1 1 1 1 1 0 0 0 1 0 0 1 0 0 1 1 ...,-25.0,23.229398,17.312336,23.229398
7,1382,23.229398,-26,8,7,1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 ...,-26.0,NaN,23.229398,NaN


# Interactive plot

In [31]:
from bokeh.plotting import figure
from bokeh.palettes import Blues
from bokeh.transform import factor_hatch, LinearColorMapper, CategoricalColorMapper
from bokeh.layouts import column, row
from bokeh.models import Range1d, RangeTool, Scatter, FixedTicker, RangeSlider, ColumnDataSource, CDSView, BooleanFilter

In [32]:
merged

,kind_x,ord,t_x,desc_x,ord_before_x,ord_after_x,kind_y,t_y,desc_y,ord_before_y,ord_after_y,sol,c,t
0,new,0,0.0000,Evaluate initial solution 0,NaN,NaN,performed,1.277930,NaN,166.0,166.0,0,NaN,Init
1,new,1,0.0000,Evaluate initial solution 1,NaN,NaN,performed,0.910061,NaN,90.0,90.0,1,NaN,Init
2,new,2,0.0000,Evaluate initial solution 2,NaN,NaN,performed,1.267810,NaN,164.0,164.0,2,NaN,Init
3,new,3,0.0000,Evaluate initial solution 3,NaN,NaN,performed,1.158110,NaN,140.0,140.0,3,NaN,Init
4,new,4,0.0000,Evaluate initial solution 4,NaN,NaN,performed,1.067500,NaN,116.0,116.0,4,NaN,Init
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3131,new,3190,27.4938,New solution on 46,NaN,NaN,performed,27.493800,NaN,3191.0,3192.0,46,NaN,Init
3132,new,3192,27.5133,New solution on 19,NaN,NaN,performed,27.513300,NaN,3193.0,3194.0,19,NaN,Init
3133,new,3194,27.5306,New solution on 17,NaN,NaN,performed,27.530600,NaN,3195.0,3196.0,17,NaN,Init
3134,new,3196,27.5446,New solution on 7,NaN,NaN,performed,27.544600,NaN,3197.0,3198.0,7,NaN,Init


In [33]:
cds = ColumnDataSource(data=dict(
    begin=merged["t_x"].to_numpy(),
    end=merged["t_y"].to_numpy(),
    desc=merged["desc_x"].to_numpy(),
    solution=merged["sol"].to_numpy(),
    element=merged["c"].fillna("X").to_numpy(),
    kind=merged["t"].to_numpy()
))

cds_a = ColumnDataSource(data=dict(
    t=archive["simulation time (s)"].to_numpy(),
))

TOOLTIPS = [
    ("index", "$index"),
    ("kind", "@kind"),
    ("solution", "@solution"),
    ("element", "@element"),
    # ("desc", "@desc"),
]

# FOS element -> color
lcm = LinearColorMapper(
    palette=Blues[256],
    low=merged["c"].min(),
    high=merged["c"].max()
)
# Kind -> color
ccm = CategoricalColorMapper(
    palette=["black", "black", "red"],
    factors=["Init", "GOM", "FI"]
)

p = figure(tools=["hover", "save"],
    tooltips = TOOLTIPS,
    x_range = (merged["t_x"].min(), merged["t_y"][min(0, len(merged["t_y"]), 1000)]),
    background_fill_color="#efefef",
    # output_backend="webgl"
    )
p.hbar(
    source=cds,
    left="begin",
    y="solution",
    right="end",
    # color={'field': 'element', 'transform': lcm},
    # line_color={'field': 'kind', 'transform': ccm},
    # hatch_pattern=factor_hatch("kind", [" ", ".", "o"], merged["t"].unique()),
    # hatch_color="white",
    height=0.9,
)
p.yaxis.ticker = FixedTicker(ticks=merged["sol"].unique())

# kind
# p.add_glyph()

sel = figure(
    height=50,
    x_range=Range1d(merged["t_x"].min(), merged["t_y"].max()),
    y_range=(-0.5, 0.5),
    y_axis_type=None,
    tools="",
    toolbar_location=None,background_fill_color="#efefef")

range_tool = RangeTool(x_range=p.x_range)
sel.add_tools(range_tool)
sel.hbar(source=cds, left=0, right=merged["t_y"].max(), color=None)
sel.circle(source=cds_a, y=0, x="t", color="red")
sel.ygrid.grid_line_color = None

# archive_wrt
show(column(sel, p))

In [20]:
from bokeh.models import DataTable, TableColumn, Slider
from bokeh.models.callbacks import CustomJS

In [ ]:
TOOLTIPS = [
    ("index", "$index"),
    ("objectives", "@objectives"),
    ("time", "@{start simulation time (s)}"),
    ("removal time", "@{end simulation time (s)}"),
]

p = figure(tools=["hover", "tap", "save", "pan", "zoom_in", "zoom_out", "wheel_zoom"], tooltips=TOOLTIPS)

time_min = archive_wrt["start simulation time (s)"].min()
time_max = archive_wrt["start simulation time (s)"].max()

cds = ColumnDataSource(archive_wrt)


# lcm = LinearColorMapper(
#     palette='Magma256',
#     low=time_min,
#     high=time_max
# )
# , color={'field': 'start simulation time (s)', 'transform': lcm}

p.circle(source=cds, x="o0", y="o1")
#  autosize_mode="fit_viewport",
dt = DataTable(source=cds, width=1000, autosize_mode="fit_columns", columns=[TableColumn(field=field) for field in cds.column_names])
sl = Slider(start=time_min, end=time_max, value=time_min, title="time")

# 
slider_changes_selection = CustomJS(
    args=dict(source=cds, sl=sl),
    code="""
    const new_sel = [];
    const v = sl.value;
    const sst = source.data["start simulation time (s)"];
    const est = source.data["end simulation time (s)"];
    for (let x = 0; x < sst.length; ++x){
        if (sst[x] <= v && (isNaN(est[x]) || est[x] > v)) {
            new_sel.push(x)
        }
    }
    source.selected.indices = new_sel;
    source.change.emit();
"""
)
sl.js_on_change('value', slider_changes_selection)

# On selection change, update slider if only a single item
update_slider_maybe = CustomJS(args=dict(source=cds, sl=sl), code="""
    if(source.selected.indices.length == 1) {
        const data = source.data;
        const simtime = data['start simulation time (s)'];
        const idx = source.selected.indices[0];
        sl.value = simtime[idx];
    }
""")
cds.selected.js_on_change('indices', update_slider_maybe)

show(column(row(column(sl), p), dt))